In [1]:
import os
import cv2
import glob
import torch
import shutil
import time
import copy
import itertools
import torch.nn as nn
import torch.optim as optim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from pathlib import Path
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image

In [2]:
# #FOR NORMAL IMAGES
# positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_positive\\'
# negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_negative\\'

In [3]:
#FOR NORMAL W/AUG IMAGES
positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\120aug_equalized_sampled_positive\\'
negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\120aug_equalized_sampled_negative\\'

In [4]:
# #FOR MASKED IMAGES
# positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\masked_images\\positive\\'
# negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\masked_images\\negative\\'

In [5]:
os.path.exists(negative_path)

True

In [6]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data\\working\\train')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data/working/test')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data/working/train/positive')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data/working/test/positive')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data/working/train/negative')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data/working/test/negative')

In [7]:
positive_train_len = int(np.floor(len(os.listdir(positive_path))*0.8))
positive_len = len(os.listdir(positive_path))

negative_train_len = int(np.floor(len(os.listdir(negative_path))*0.8))
negative_len = len(os.listdir(negative_path))

In [8]:
for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive'):
    os.remove(file.path)
for trainimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive')
    
for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative'):
    os.remove(file.path)
for trainimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative')

for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/positive'):
    os.remove(file.path)
for testimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len, positive_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/positive')

for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/negative'):
    os.remove(file.path)
for testimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len, negative_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/negative')

In [9]:
normalizer = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        normalizer
    ]),
    
    'validation': transforms.Compose([
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        normalizer
    ]),

    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        normalizer
    ])
}

In [10]:
data_images = {
    'train': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/train', data_transforms['train']),
    'validation': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/val', data_transforms['validation']),
    'test': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/test', data_transforms['test'])
}

In [11]:
dataloaders = {
    'train': torch.utils.data.DataLoader(data_images['train'], batch_size=32, shuffle=True, num_workers=0),
    'validation': torch.utils.data.DataLoader(data_images['validation'], batch_size=32,shuffle=True,num_workers=0),
    'test': torch.utils.data.DataLoader(data_images['test'], batch_size=32,shuffle=True,num_workers=0)
}

In [12]:
device_id = 0
torch.cuda.device_count()
torch.cuda.get_device_name(device_id)
# print(device_id)

'NVIDIA GeForce GTX 1660 Ti'

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
if torch.cuda.is_available():
    model.cuda()

In [14]:
for param in model.parameters():
    param.requires_grad = False

In [15]:
model.fc = nn.Sequential(
    nn.Linear(2048, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 3)
).to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [17]:
def trained_model(model, criterion, optimizer, epochs):
    since = time.time()

    best_acc = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0

    for epoch in range(epochs):
        
        print('Epoch:', str(epoch+1) + '/' + str(epochs))
        print('-'*10)
        
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train() #this trains the model
            else:
                model.eval() #this evaluates the model

            running_loss, running_corrects = 0.0, 0 

            iteration_count=0
            total_precision=0
            total_recall=0
            total_f1=0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) #convert inputs to cpu or cuda
                labels = labels.to(device) #convert labels to cpu or cuda

                outputs = model(inputs) #outputs is inputs being fed to the model
                loss = criterion(outputs, labels) #outputs are fed into the model

                if phase == 'train':
                    optimizer.zero_grad() #sets gradients to zero
                    loss.backward() #computes sum of gradients
                    optimizer.step() #preforms an optimization step

                _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
                running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
                running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

                iteration_count+=1
                precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                total_precision+=precision_it
                total_recall+=recall_it
                total_f1+=f1_it

            epoch_loss = running_loss / len(data_images[phase]) #this is the epoch loss
            epoch_acc = running_corrects.double() / len(data_images[phase]) #this is the epoch accuracy
            epoch_precision=total_precision/iteration_count
            epoch_recall=total_recall/iteration_count
            epoch_f1=total_f1/iteration_count

            print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
                phase, epoch_loss, epoch_acc, epoch_precision, epoch_recall, epoch_f1))

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_precision = epoch_precision
                best_recall = epoch_recall
                best_f1 = epoch_f1


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))

    return model

In [18]:
model = trained_model(model, criterion, optimizer, 30)

Epoch: 1/30
----------


C:\Users\arnav\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.356 Acc: 0.848 Prec: 0.866 Recall: 0.848 F1 0.838
test Loss: 0.275 Acc: 0.918 Prec: 0.931 Recall: 0.916 F1 0.917
Epoch: 2/30
----------
train Loss: 0.242 Acc: 0.896 Prec: 0.914 Recall: 0.896 F1 0.896
test Loss: 0.587 Acc: 0.682 Prec: 0.828 Recall: 0.679 F1 0.665
Epoch: 3/30
----------
train Loss: 0.227 Acc: 0.903 Prec: 0.922 Recall: 0.904 F1 0.903
test Loss: 0.316 Acc: 0.866 Prec: 0.906 Recall: 0.864 F1 0.865
Epoch: 4/30
----------
train Loss: 0.186 Acc: 0.928 Prec: 0.938 Recall: 0.928 F1 0.929
test Loss: 0.360 Acc: 0.839 Prec: 0.888 Recall: 0.838 F1 0.837
Epoch: 5/30
----------
train Loss: 0.180 Acc: 0.930 Prec: 0.940 Recall: 0.930 F1 0.930
test Loss: 0.496 Acc: 0.757 Prec: 0.855 Recall: 0.760 F1 0.756
Epoch: 6/30
----------
train Loss: 0.169 Acc: 0.935 Prec: 0.945 Recall: 0.935 F1 0.935
test Loss: 0.217 Acc: 0.921 Prec: 0.932 Recall: 0.922 F1 0.922
Epoch: 7/30
----------
train Loss: 0.175 Acc: 0.935 Prec: 0.946 Recall: 0.935 F1 0.935
test Loss: 0.507 Acc: 0.750 Prec: 0.

In [19]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_data/working/models')

In [20]:
torch.save(model.state_dict(), 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_weights/working/models/weights.h5') #save the model's weights
model.load_state_dict(torch.load('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\resnet50_model_weights/working/models/weights.h5')) #load the model's weights

<All keys matched successfully>